### Import the libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
from io import StringIO

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### Load the dataset

In [2]:
data = {
    "state": ["CA", "WA", "CA", np.nan, "NV", "WA"],
    "gender": ["male", "female", "female", "male", np.nan, "female"],
    "age": [34, 29, 22, 44, 55, np.nan],
    "weight": [122, 150, 130, 157, 140, 175],
    "target": [0, 1, 0, 1, 0, 1]
}
df = pd.DataFrame(data)

In [3]:
df

,state,gender,age,weight,target
0,CA,male,34.0,122,0
1,WA,female,29.0,150,1
2,CA,female,22.0,130,0
3,NaN,male,44.0,157,1
4,NV,NaN,55.0,140,0
5,WA,female,NaN,175,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   state   5 non-null      object 
 1   gender  5 non-null      object 
 2   age     5 non-null      float64
 3   weight  6 non-null      int64  
 4   target  6 non-null      int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 372.0+ bytes


* state - Categorical Column (Nominal) - Has missing values as well - SimpleImputer, OneHotEncoder
* gender - Categorical Column (Nominal) - Has missing values as well - SimpleImputer, OneHotEncoder
* age - Numerical Column - Has missing values as well - SimpleImputer, StandardScaler
* weight - Numerical Column - StandardScaler

### Train Test Split

In [5]:
X, y = df.drop('target', axis=1), df['target'] 

In [6]:
X.shape, y.shape

((6, 4), (6,))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### ColumnTransformer

In [8]:
numeric_preprocessor = Pipeline(
    steps=[
        ("imputation_mean", SimpleImputer(missing_values=np.nan, strategy="mean")),
        ("scaler", StandardScaler()),
    ]
)

categorical_preprocessor = Pipeline(
    steps=[
        ("imputation_constant", SimpleImputer(fill_value="missing", strategy="constant")),
        ("onehot", OneHotEncoder(handle_unknown="ignore")),
    ]
)

preprocessor = ColumnTransformer(
    [
        ("categorical", categorical_preprocessor, ["state", "gender"]),
        ("numerical", numeric_preprocessor, ["age", "weight"]),
    ]
)

### Pipeline

In [9]:
pipe = Pipeline(
    steps=[
        ('Preprocessor', preprocessor),
        ('Classifier', LogisticRegression())
    ]
)

In [10]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('Preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('imputation_constant',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['state', 'gender']),
                                                 ('numerical',
                                                  Pipeline(steps=[('imputation_mean',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'weight'])])),
                ('Classifier', LogisticRegression())])

### Prediction

In [11]:
y_pred = pipe.predict(X_test)

In [12]:
accuracy_score(y_test, y_pred)

1.0